# Deep Learning Analysis for Non-Small Cell Lung Cancer

In [ ]:
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
import lightning as L
